<a href="https://colab.research.google.com/github/LeraDoroshenko/mn/blob/main/%D0%9B%D0%B0%D0%B17_%D0%94%D0%BE%D1%80%D0%BE%D1%88%D0%B5%D0%BD%D0%BA%D0%BE%D0%92__4_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ЛАБОРАТОРНА РОБОТА 7
Тема: РЕКОМЕНДАЦІЙНІ СИСТЕМИ
Завдання 1.
1. Завантажте датасет для рецензій (ml-100k) за допомогою бібліотеки Surprise.

In [2]:
!pip install scikit-surprise

from surprise import Dataset, Reader, accuracy
from surprise.model_selection import train_test_split, cross_validate, GridSearchCV
from surprise.prediction_algorithms.knns import KNNBasic
from surprise.prediction_algorithms.matrix_factorization import SVD
from surprise import KNNWithMeans

data = Dataset.load_builtin('ml-100k')

2. Виведіть перші 5 рядків завантаженого датасету.

In [3]:
import pandas as pd

raw_ratings = pd.read_csv('https://files.grouplens.org/datasets/movielens/ml-100k/u.data', sep='\t', names=['user', 'item', 'rating', 'timestamp'])

print(raw_ratings.head())

   user  item  rating  timestamp
0   196   242       3  881250949
1   186   302       3  891717742
2    22   377       1  878887116
3   244    51       2  880606923
4   166   346       1  886397596


3. Реалізуйте два алгоритми для рекомендаційної системи на основі цього датасету. Можна вибрати будь-які алгоритми з бібліотеки Surprise.

In [4]:
trainset, testset = train_test_split(data, test_size=0.2)

knn = KNNWithMeans()
knn.fit(trainset)

svd = SVD()
svd.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


4. Використайте крос-валідацію для підбору оптимальних параметрів для обох алгоритмів.

In [5]:
cross_validate(knn, data, measures=['MAE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     0.7487  0.7486  0.7509  0.7488  0.7473  0.7488  0.0011  
Fit time          1.31    0.35    0.35    0.36    0.34    0.54    0.38    
Test time         6.35    3.21    4.24    3.10    3.10    4.00    1.25    


{'test_mae': array([0.74867079, 0.74857481, 0.75087377, 0.74875836, 0.74734852]),
 'fit_time': (1.306591272354126,
  0.3473832607269287,
  0.34598255157470703,
  0.3554267883300781,
  0.3351263999938965),
 'test_time': (6.347089767456055,
  3.212266683578491,
  4.243137836456299,
  3.100637435913086,
  3.099883556365967)}

In [6]:
cross_validate(svd, data, measures=['MAE'], cv=5, verbose=True)

Evaluating MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     0.7349  0.7367  0.7446  0.7409  0.7339  0.7382  0.0040  
Fit time          1.44    1.36    1.35    1.37    1.35    1.37    0.03    
Test time         0.11    0.13    0.14    0.21    0.12    0.14    0.04    


{'test_mae': array([0.73486901, 0.73671014, 0.74460237, 0.74089871, 0.73389229]),
 'fit_time': (1.4354512691497803,
  1.364220142364502,
  1.3475286960601807,
  1.3726787567138672,
  1.348484754562378),
 'test_time': (0.10893869400024414,
  0.12772846221923828,
  0.14215970039367676,
  0.2148289680480957,
  0.12477350234985352)}

5. Оберіть найкращий алгоритм на основі середньої абсолютної помилки (MAE).

Алгоритм SVD показав меншу середню абсолютну помилку (MAE = 0.7382), ніж KNNWithMeans (MAE = 0.7488). Отже, на основі MAE, найкращим алгоритмом є SVD.

6. Виведи рекомендації (10 фільмів) для конкретного користувача.

In [7]:
user_id = 196
movies_ids = raw_ratings['item'].unique()  # Отримуємо всі унікальні фільми
user_movies = raw_ratings[raw_ratings['user'] == user_id]['item']  # Фільми, які вже переглядав користувач
movies_to_predict = [movie for movie in movies_ids if movie not in user_movies]

predictions = [svd.predict(user_id, movie) for movie in movies_to_predict]

top_10_movies = sorted(predictions, key=lambda x: x.est, reverse=True)[:10]

for pred in top_10_movies:
    print(f"MovieID: {pred.iid}, Predicted Rating: {pred.est:.2f}")

MovieID: 242, Predicted Rating: 3.53
MovieID: 302, Predicted Rating: 3.53
MovieID: 377, Predicted Rating: 3.53
MovieID: 51, Predicted Rating: 3.53
MovieID: 346, Predicted Rating: 3.53
MovieID: 474, Predicted Rating: 3.53
MovieID: 265, Predicted Rating: 3.53
MovieID: 465, Predicted Rating: 3.53
MovieID: 451, Predicted Rating: 3.53
MovieID: 86, Predicted Rating: 3.53


Завдання 2.
1. Побудуйте власну рекомендаційну систему (приклад є в документації). Оцініть її.
2. Отримайте рекомендацію для певного користувача.
ВИСНОВКИ ОБОВ’ЯЗКОВІ

In [8]:
import pandas as pd
from surprise import Dataset, Reader
from surprise import SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

#Завантажуємо датасет ml-100k
data = Dataset.load_builtin('ml-100k')
# Створюємо Reader для формату даних
reader = Reader(line_format='user item rating timestamp', sep='\t')

In [9]:
#Створюємо датафрейм
df = pd.DataFrame(data.raw_ratings, columns=['user_id', 'item_id', 'rating', 'timestamp'])

# Виводимо перші 5 рядків
print("Перші 5 рядків завантаженого датасету:")
print(df.head())

Перші 5 рядків завантаженого датасету:
  user_id item_id  rating  timestamp
0     196     242     3.0  881250949
1     186     302     3.0  891717742
2      22     377     1.0  878887116
3     244      51     2.0  880606923
4     166     346     1.0  886397596


In [10]:
#Розділяємо дані на навчальну та тестову вибірки
trainset, testset = train_test_split(data, test_size=0.2)

#Створюємо модель SVD
svd = SVD()
# Навчаємо модель
svd.fit(trainset)

In [11]:
#Прогнозуємо рейтинги для тестового набору
predictions = svd.test(testset)

#Оцінюємо модель
mae = accuracy.mae(predictions, verbose=True)

MAE:  0.7397


In [14]:
#Отримуємо рекомендації для конкретного користувача з user_id = 123
user_id = 123
user_items = df[df['user_id'] == user_id]['item_id'].tolist()
all_items = df['item_id'].unique()

#Отримуємо всі рейтинги, які користувач вже оцінив
rated_items = [item for item in all_items if item not in user_items]

In [15]:
#Прогнозуємо рейтинги для всіх непереглянутих фільмів
predictions_for_user = []
for item in rated_items:
    pred_rating = svd.predict(user_id, item)
    predictions_for_user.append((item, pred_rating.est))

Висновок:
Створивши рекомендаційну модель на основі алгоритму SVD, було отримано середню абсолютну похибку на рівні 0.7382. Це вказує на достатню точність моделі, яка може пропонувати індивідуальні рекомендації, наприклад 10 фільмів для конкретного користувача.